<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [9]:
%%capture
!pip3 install folium
!pip3 install wget
!pip3 install pandas

In [10]:
import folium
import wget
import pandas as pd

In [11]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [17]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [19]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [22]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [24]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


In [28]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    site_name = row['Launch Site']
    
    # Añadir círculo
    folium.Circle(
        location=coordinate,
        radius=100,
        color='red',
        opacity=0.3,
        fill=True,
        fill_color='#red',
        fill_opacity=0.1
    ).add_child(folium.Popup(site_name)).add_to(site_map)
    
    # Añadir marcador con etiqueta del nombre del sitio
    html_label = '<div style="font-size: 12; color:#d35400;"><b>{}</b></div>'.format(site_name)
    marker = folium.map.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=html_label
        )
    )
    marker.add_to(site_map)

site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


**Are all launch sites near the Equator?**

- No. Florida sites are at ~28.5°N, and VAFB is at ~34.6°N. None are very close to the Equator.

**Are all launch sites near the coast?**

Yes. All sites are relatively close to the ocean, which is important for safety during launches.

**Explanation**

- Coastal locations reduce risk to populated areas.

- Lower latitudes (like Florida) help gain extra rotational speed from the Earth, useful for certain orbits.

# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [35]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [39]:
marker_cluster = MarkerCluster()


_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [41]:
# Función para asignar color con 75% de transparencia
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'rgba(0, 128, 0, 0.75)'   # verde semi-transparente
    else:
        return 'rgba(255, 0, 0, 0.75)'   # rojo semi-transparente

# Aplicar la función a la columna 'class' para crear 'marker_color'
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

# Verificar las últimas filas
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,"rgba(0, 128, 0, 0.75)"
47,KSC LC-39A,28.573255,-80.646895,1,"rgba(0, 128, 0, 0.75)"
48,KSC LC-39A,28.573255,-80.646895,1,"rgba(0, 128, 0, 0.75)"
49,CCAFS SLC-40,28.563197,-80.576820,1,"rgba(0, 128, 0, 0.75)"
50,CCAFS SLC-40,28.563197,-80.576820,1,"rgba(0, 128, 0, 0.75)"
51,CCAFS SLC-40,28.563197,-80.576820,0,"rgba(255, 0, 0, 0.75)"
52,CCAFS SLC-40,28.563197,-80.576820,0,"rgba(255, 0, 0, 0.75)"
53,CCAFS SLC-40,28.563197,-80.576820,0,"rgba(255, 0, 0, 0.75)"
54,CCAFS SLC-40,28.563197,-80.576820,1,"rgba(0, 128, 0, 0.75)"
55,CCAFS SLC-40,28.563197,-80.576820,0,"rgba(255, 0, 0, 0.75)"


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [43]:
# Crear un MarkerCluster
marker_cluster = MarkerCluster()

# Añadir el MarkerCluster al mapa existente (que ya tiene los círculos y nombres de la Tarea 1)
site_map.add_child(marker_cluster)

# Para cada registro de lanzamiento, crear un marcador con color según 'marker_color'
for index, row in spacex_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    marker = folium.Marker(
        location=coordinate,
        icon=folium.Icon(color='white', icon_color=row['marker_color']),
        popup=f"Launch Site: {row['Launch Site']}<br>Class: {row['class']}"
    )
    marker_cluster.add_child(marker)

# Mostrar mapa con todo lo anterior más los marcadores de lanzamientos
site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [51]:
from folium.plugins import MousePosition

# Formato de las coordenadas que se mostrarán
formatter = "function(num) {return L.Util.formatNum(num, 5);};"

# Crear MousePosition
mouse_position = MousePosition(
    position='topright',        # esquina superior derecha
    separator=' Long: ',        # separador entre latitud y longitud
    empty_string='NaN',         # valor por defecto
    lng_first=False,            # latitud primero
    num_digits=20,              # número de decimales
    prefix='Lat:',              # prefijo antes de la latitud
    lat_formatter=formatter,
    lng_formatter=formatter,
)

# Añadir MousePosition al mapa existente
site_map.add_child(mouse_position)

# Mostrar mapa con MousePosition agregado
site_map


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [54]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [56]:
# Ejemplo: coordenadas del sitio de lanzamiento
launch_site_lat = 28.563197  # CCAFS SLC-40
launch_site_lon = -80.576820

# Coordenadas del punto más cercano de la costa (obtenido con MousePosition)
coastline_lat = 28.56445
coastline_lon = -80.56852

# Calcular la distancia entre el sitio de lanzamiento y la costa
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

print(f"Distance from launch site to closest coastline: {distance_coastline:.2f} km")

Distance from launch site to closest coastline: 0.82 km


_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [58]:
# Coordenadas del punto de la costa
coastline_coord = [coastline_lat, coastline_lon]

# Crear y añadir un marcador mostrando la distancia
distance_marker = folium.Marker(
    location=coastline_coord,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>{}</b></div>'.format("{:10.2f} KM".format(distance_coastline))
    )
)

# Añadir el marcador al mapa existente
distance_marker.add_to(site_map)

# Mostrar mapa actualizado
site_map



_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [99]:
# Coordenadas del sitio de lanzamiento y del punto de la costa
launch_site_coord = [launch_site_lat, launch_site_lon]
coastline_coord = [coastline_lat, coastline_lon]

# Crear la lista de coordenadas para la línea
coordinates = [launch_site_coord, coastline_coord]

# Crear y añadir PolyLine
lines = folium.PolyLine(
    locations=coordinates,
    weight=2,        # grosor de la línea
    color='blue',    # color de la línea
    opacity=0.5      # transparencia
)
site_map.add_child(lines)

# Mostrar mapa actualizado
site_map



Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [110]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
# Coordenadas
highway_coord = [28.56315, -80.57074]
railroad_coord = [28.56489, -80.58663]
city_coord = [28.08023, -80.60798]
launch_site_coord = [launch_site_lat, launch_site_lon]

# Calcular distancias
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_coord[0], highway_coord[1])
distance_railroad = calculate_distance(launch_site_lat, launch_site_lon, railroad_coord[0], railroad_coord[1])
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_coord[0], city_coord[1])

# Crear y añadir marcadores con DivIcon mostrando la distancia
highway_marker = folium.Marker(
    location=highway_coord,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12; color:#1f77b4;"><b>{distance_highway:.2f} KM</b></div>'
    )
)
railroad_marker = folium.Marker(
    location=railroad_coord,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12; color:#ff7f0e;"><b>{distance_railroad:.2f} KM</b></div>'
    )
)
city_marker = folium.Marker(
    location=city_coord,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12; color:#2ca02c;"><b>{distance_city:.2f} KM</b></div>'
    )
)

# Añadir marcadores al mapa
highway_marker.add_to(site_map)
railroad_marker.add_to(site_map)
city_marker.add_to(site_map)

# Dibujar líneas entre el sitio de lanzamiento y cada punto
lines_highway = folium.PolyLine(locations=[launch_site_coord, highway_coord], color='blue', weight=2, opacity=0.5)
lines_railroad = folium.PolyLine(locations=[launch_site_coord, railroad_coord], color='blue', weight=2, opacity=0.5)
lines_city = folium.PolyLine(locations=[launch_site_coord, city_coord], color='blue', weight=2, opacity=0.5)

site_map.add_child(lines_highway)
site_map.add_child(lines_railroad)
site_map.add_child(lines_city)

# Mostrar mapa actualizado
site_map


After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

Also please try to explain your findings.




**1. Are launch sites in close proximity to railways?**
Yes, the launch site is relatively close to the railway. The distance calculated shows only a few kilometers, which suggests that rail infrastructure is nearby, likely for logistics purposes.

**2. Are launch sites in close proximity to highways?**
Yes, the launch site is very close to the highway. The short distance indicates easy access for vehicles transporting materials, personnel, or payloads.

**3. Are launch sites in close proximity to coastline?**
Yes, launch sites are near the coastline. This is typical for rocket launch facilities, as proximity to the ocean allows rockets to fly over water during initial ascent, reducing risk to populated areas.

**4. Do launch sites keep certain distance away from cities?**
Yes, launch sites are located at a significant distance from cities. This ensures safety in case of launch failures and minimizes disturbance to urban populations.

**Summary:**
From the plotted distances, it is clear that launch sites are strategically located: close enough to transport infrastructure (highways and railways) and the coastline for operational efficiency, while being safely distant from cities t want me to do that?


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
